In [12]:
# importing modules

import numpy as np
np.__version__
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
import pickle
import joblib
import os
import gc
import seaborn as sb
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix


physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

tf.test.gpu_device_name()
tf.device('/GPU:0')

2024-11-28 17:01:18.135783: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-28 17:01:18.135845: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3070 Ti Laptop GPU computeCapability: 8.6
coreClock: 1.035GHz coreCount: 46 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 357.69GiB/s
2024-11-28 17:01:18.135931: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-28 17:01:18.135951: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-28 17:01:18.13

In [6]:
num_classes = 19 #0 - 18
np.random.seed(441) #setting seed
img_rows, img_cols = 28, 28 #image dimensions

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

X = np.concatenate([x_train, x_test])
Y = np.concatenate([y_train, y_test])

X = X.astype('float32')
X /= 255

def dataset(X, Y, output=100000):
    pairs_per_combination = output//100
    imgs = []
    labels = []
    l1 = []
    l2 = []
    sample_idx=0
    for i in range(10):
        for j in range(10):
            idxl1 = np.where(Y == i)[0]
            idxl2 = np.where(Y == j)[0]
            for k in range(pairs_per_combination):
                idx1 = np.random.choice(idxl1)
                idx2 = np.random.choice(idxl2)
                combined_img = np.hstack((X[idx1], X[idx2]))
                combined_label = i+j
                imgs.append(combined_img)
                labels.append(combined_label)
                l1.append(i)
                l2.append(j)
            sample_idx += 1
    shuffle_idx = np.random.permutation((len(imgs)))
    imgs = np.array(imgs)[shuffle_idx]
    labels = np.array(labels)[shuffle_idx]
    l1 = np.array(l1)[shuffle_idx]
    l2 = np.array(l2)[shuffle_idx]
    return imgs, labels, l1, l2

X, Y, D1, D2 = dataset(X, Y)

X = X.reshape(X.shape[0], -1)

In [7]:
train_ratio = 0.8
test_ratio = 0.2

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=(1-train_ratio), random_state=441)

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

x_train shape: (80000, 1568)
x_test shape: (20000, 1568)
y_train shape: (80000,)
80000 train samples
20000 test samples


In [8]:
with open('x_train_inf.pickle', 'wb') as f:
    pickle.dump(x_train, f)

with open('y_train_inf.pickle', 'wb') as f:
    pickle.dump(y_train, f)
    
with open('x_test_inf.pickle', 'wb') as f:
    pickle.dump(x_test, f)
    
with open('y_test_inf.pickle', 'wb') as f:
    pickle.dump(y_test, f)

In [9]:
with open('x_train_inf.pickle', 'rb') as f:
    x_train = pickle.load(f)

with open('y_train_inf.pickle', 'rb') as f:
    y_train = pickle.load(f)
    
with open('x_test_inf.pickle', 'rb') as f:
    x_test = pickle.load(f)
    
with open('y_test_inf.pickle', 'rb') as f:
    y_test = pickle.load(f)

In [3]:
svcClass = LinearSVC(penalty='l2', dual=False, C = 1, multi_class='ovr', verbose=1)
svcClass.fit(x_train, np.argmax(y_train, axis=1))
svcPred = svcClass.predict(x_test)
score = svcClass.score(x_test, np.argmax(y_test, axis=1))
print(score)

[LibSVM]*
optimization finished, #iter = 721
obj = -90.069786, rho = 0.290804
nSV = 342, nBSV = 55
*.*
optimization finished, #iter = 1138
obj = -209.600778, rho = 0.954583
nSV = 623, nBSV = 179
*.*
optimization finished, #iter = 1175
obj = -188.689267, rho = 0.870304
nSV = 625, nBSV = 160
*.*
optimization finished, #iter = 1222
obj = -167.297423, rho = 0.881995
nSV = 610, nBSV = 128
*.*
optimization finished, #iter = 1339
obj = -240.379443, rho = 0.941149
nSV = 722, nBSV = 212
*.*
optimization finished, #iter = 1178
obj = -213.699651, rho = 0.991493
nSV = 657, nBSV = 182
*.*
optimization finished, #iter = 1164
obj = -179.272563, rho = 0.996559
nSV = 644, nBSV = 153
*.*
optimization finished, #iter = 1360
obj = -215.828615, rho = 0.964271
nSV = 747, nBSV = 197
*.*
optimization finished, #iter = 1323
obj = -188.175579, rho = 0.984647
nSV = 718, nBSV = 162
*
optimization finished, #iter = 930
obj = -105.320860, rho = 0.959269
nSV = 541, nBSV = 78
*
optimization finished, #iter = 964
obj 

In [8]:
with open("svm_model.pkl", "wb") as f:
    pickle.dump(svcClass, f)

In [6]:
loaded = pickle.load(open("svm_model.pkl", "rb"))

In [7]:
svcPred = loaded.predict(x_test)
score = loaded.score(x_test, np.argmax(y_test, axis=1))
print(score)

0.6916


In [ ]:
confusion = confusion_matrix(y_true=np.argmax(y_test, axis=1), y_pred=svcPred)
plt.subplots(figsize=(15, 8))
sb.heatmap(confusion, annot=True, fmt='g')

In [ ]:
forestClass = RandomForestClassifier(max_depth=200)
forestClass.fit(x_train, np.argmax(y_train, axis=1))
forestPred = forestClass.predict(x_test)
score = forestClass.score(x_test, np.argmax(y_test, axis=1))
print(score)

In [ ]:
with open("forest.pkl", 'wb') as f:
    pickle.dump(forestClass, f)

In [ ]:
forestPred = loaded.predict(x_test)
score = loaded.score(x_test, np.argmax(y_test, axis=1))
print(score)

In [ ]:
confusion = confusion_matrix(y_true=np.argmax(y_test, axis=1), y_pred=forestPred)
plt.subplots(figsize=(15, 8))
sb.heatmap(confusion, annot=True, fmt='g')